# What is overloading?

- In other languages, we can have multiple functions assigned to a single name
    - As long as the signatures are different
        - i.e. the data types of our parameters determines which function to use

- This doesn't exist in Python since we don't have static typing
    - Therefore, function signatures aren't really a thing

- Instead, we can use **single dispatch generic functions**
    - Allows us to something similar

# How do we use single dispatch generic functions?

- Let's consider an example where we're translating HTML

In [2]:
from html import escape

In [3]:
def html_escape(arg):
    return escape(str(arg))

In [4]:
def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

In [5]:
def html_real(a):
    return '{0:.2f}'.format(round(a, 2))

In [6]:
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

In [7]:
def html_list(l):
    items = (f'<li>{html_escape(i)}<\li>' for i in l)
    
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [8]:
def html_dict(d):
    items = (f'<li>{k}={v}<\li>' for k,v in d.items())
    
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [9]:
var = """
this
is
a
multi
line string with special
characters: 10 < 100
"""

In [10]:
print(html_str(var))

<br/>
this<br/>
is<br/>
a<br/>
multi<br/>
line string with special<br/>
characters: 10 &lt; 100<br/>



In [11]:
print(html_int(255))

255(<i>0xff</i>)


In [12]:
print(html_escape(3+10j))

(3+10j)


- We want to be able to support Decimals

In [13]:
from decimal import Decimal

- Now, we'll write our dispatch function

In [14]:
def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        return html_escape(arg)

In [15]:
htmlize(100)

'100(<i>0x64</i>)'

In [16]:
htmlize(var)

'<br/>\nthis<br/>\nis<br/>\na<br/>\nmulti<br/>\nline string with special<br/>\ncharacters: 10 &lt; 100<br/>\n'

# Will we run into issues with our code so far?

- Yes
    - There are actual problems with our code
    - There are also prblems with how we're solving our problem

In [17]:
print(htmlize([1,2,3]))

<ul>
<li>1<\li>
<li>2<\li>
<li>3<\li>
</ul>


- That worked fine
    - But what if our list contained both multi-line text, a tuple of integers, and an integer

In [19]:
print(htmlize(["""
h
e
l
l
o""", (1,2), 100]))

<ul>
<li>
h
e
l
l
o<\li>
<li>(1, 2)<\li>
<li>100<\li>
</ul>


- This isn't right
    - The elements of the list need to be `htmlize`-d themselves
        - This means we need to change the code to:

In [20]:
def html_list(l):
    items = (f'<li>{htmlize(i)}<\li>' for i in l)
    
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

- **But wait a minute!**
    - *`htmlize` was originally defined **after** `html_list`
        - *Won't this cause an error?*
            - NO

- In Python, this is allowed

____

**Example**

In [21]:
def func_defined_first(a):
    return func_defined_second(a)

def func_defined_second(a):
    return f'afadsfsdaf {a}'

In [22]:
func_defined_first(10)

'afadsfsdaf 10'

- As we can see, even though `func_defined_second` is called in `func_defined_first` before it's even defined, it's alright
    - **Only needs to be defined by the time it's called - NOT when it's defined**
    
___

- But our approach is gonna cause problems
    - Instead, we'd like `htmlize` to maintain a dictionary of functions
        - To prove that we don't need to define them first, we can delete the previously defined functions

In [23]:
del html_escape, html_int, html_real, html_str, html_list, html_dict

- **Note**: if we have some dict `d`, then `d.get(key, alt)` gives us the value from the dict corresponding to the key
    - If the key isn't found in `d`, it returns the `alt` value
    
___

**Example**

In [24]:
d = {1:'a', 2:'b'}

In [26]:
d.get(1,'alt'), d.get(2,'alt'), d.get(3,'alt')

('a', 'b', 'alt')

____

In [28]:
def htmlize(arg):
    registry = {object: html_escape, 
                int: html_int, 
                float: html_real,
                Decimal: html_real, 
                str: html_str, 
                list: html_list, 
                tuple: html_tuple, 
                set: html_set, 
                dict: html_dict}
    
    fn = registry.get(type(arg), registry[object])
    return fn(arg)

# Will this solution cause problems?

- Possibly
    - It's not ideal to hard-code the `registry` dictionary
        - We can find a smarter solution

- First, we'll create a new decorator that creates a dispatcher function

In [29]:
def single_argument_dispatch(fn):
    registry = {}
    # Setting the default function
    registry[object] = fn
    
    def inner(arg):
        return registry[object](arg)
    
    return inner

- *What is this decorator even doing?*
    1. Takes the function we're defining
    2. Creates a dictionary called `registry`
    3. Sets our function from 1 as the default function in registry
    4. Defines an `inner` function that simply runs `fn` on `arg`
    5. Returns the `inner` function

In [30]:
@single_argument_dispatch
def htmlize(a):
    return escape(str(a))

In [31]:
htmlize('1 < 100')

'1 &lt; 100'

- Here, when we called `htmlize`, it actually called `inner`, which simply runs `escape(str('1 < 100'))`

- Let's expand our decorator

In [36]:
def single_argument_dispatch(fn):
    registry = {}
    # Setting the default function
    registry[object] = fn
    registry[int] = lambda a: f'{a}(<i>{str(hex(a))}</i>)'
    registry[str] = lambda s: escape(s).replace('\n', '<br/>\n')
    
    def inner(arg):
        f = registry.get(type(arg), registry[object])
        return f(arg)
    
    return inner

- Now, our registry contains two extra key-value pairs

In [37]:
@single_argument_dispatch
def htmlize(a):
    return escape(str(a))

In [38]:
htmlize('1 < 100')

'1 &lt; 100'

- Now, let's try it for an `int`

In [39]:
htmlize(100)

'100(<i>0x64</i>)'

- Ok, looks like it worked

- *But how is this different from our previous `htmlize` function?*
    - We still need to hard-code the lambda functions
        - Our solution should **feed in functions from the outside**

- New decorator:

In [45]:
def single_argument_dispatch(fn):
    registry = {}
    # Setting the default function
    registry[object] = fn
    
    def decorated_function(arg):
        f = registry.get(type(arg), registry[object])
        return f(arg)
    
    # Creating a decorator factory INSIDE our decorator
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    return decorated_function

In [46]:
@single_argument_dispatch
def htmlize(a):
    return escape(str(a))

In [48]:
htmlize('1 < 100')

'1 &lt; 100'

- It worked as expected, but let's try an `int`

In [49]:
htmlize(100)

'100'

- That didn't format it at all
    - Because we never added it to the `registry`

- *How can we add it to the `registry`?*

In [50]:
def single_argument_dispatch(fn):
    registry = {}
    # Setting the default function
    registry[object] = fn
    
    def decorated_function(arg):
        f = registry.get(type(arg), registry[object])
        return f(arg)
    
    # Creating a decorator factory INSIDE our decorator
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    # Adding register as a method of the decorated function
    decorated_function.register = register
    
    return decorated_function

- **Recall**: we can set one function as a method of another

___

**Example**

In [51]:
d = lambda x: x
e = lambda y: y+1
d.e = e

In [53]:
d(1), e(1), d.e(1)

(1, 2, 2)

____

In [54]:
@single_argument_dispatch
def htmlize(a):
    return escape(str(a))

- Our new version of `htmlize` has the built-in method `register`

In [55]:
htmlize.register

<function __main__.single_argument_dispatch.<locals>.register(type_)>

### Recap

1. *Why did we define `htmlize` to simply return `escape(str(a))`?*
    - Because that's the most basic thing it needs to do
        - **That's why we set `registry[object] = fn` in our decorator**
            - If the `arg` fed into it is some generic object, that's what runs
2. *What about the other `htmlize` functions?*
    - We created our decorator to enable them to be added later

**Line-by-line review**

```python
def single_argument_dispatch(fn):
    registry = {}
    # Setting the default function
    registry[object] = fn
```

- So far, our decorator:
    1. Takes the function we want to decorate
        - In this case, it'll be the basic version of `htmlize`
    2. Creates an empty dictionary called `registry`
        - **Note**: this registry will be in the non-local scope of our `htmlize` function
            - i.e. when we define the function and decorate it, the `registry` will forever be associated with the function
    3. Defines our default function in the registry as the function we fed in

```python
    # Creating a decorator factory INSIDE our decorator
    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
```

- This part is confusing
    - It creates a decorator that we feed in the data type before the definition
        - Now, whatever function we define gets added to the `registry`
        
        
```python
    # Adding register as a method of the decorated function
    decorated_function.register = register
    
    return decorated_function
```

- This simply associates our new decorator with the function

- Let's add a new function to the registry

In [56]:
@htmlize.register(int)
def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

- Now, let's test it

In [57]:
htmlize(100)

'100(<i>0x64</i>)'

- It worked!

In [58]:
@htmlize.register(list)
def html_list(l):
    items = (f'<li>{htmlize(i)}<\li>' for i in l)
    
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

- **Recall**: this is the same as:

```python
html_list = htmlize.register(list)(html_list)
```

- But we didn't notice something - we can stack the decorators!

In [59]:
@htmlize.register(tuple)
@htmlize.register(list)
def html_list(l):
    items = (f'<li>{htmlize(i)}<\li>' for i in l)
    
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [60]:
htmlize([1,2,3])

'<ul>\n<li>1(<i>0x1</i>)<\\li>\n<li>2(<i>0x2</i>)<\\li>\n<li>3(<i>0x3</i>)<\\li>\n</ul>'

# Will looking up the type of `arg` cause problems?

- Yes
    - Consider `True`

In [61]:
htmlize(True)

'True'

- Ideally, this would recognize `True` as an integral number

- Let's try something new

In [62]:
from numbers import Integral

In [65]:
isinstance(True, Integral)

True

- *What's the difference between `isinstance` and looking up the type?*
    - It recognizes sub-class types
    
___

**Example**

In [66]:
class Person:
    pass

In [67]:
class Student(Person):
    pass

In [68]:
p = Student()

In [69]:
type(p)

__main__.Student

- Recognizes the object type

In [70]:
isinstance(p, Person)

True

- Also recognizes that it's a `Person`
    - A `Student` is also a `Person` though inheritance
        - This wouldn't be recognized if we only checked the type

___

- Now, let's say we want to define a function for integral numbers
    - First, let's redefine our `htmlize` function

In [75]:
@single_argument_dispatch
def htmlize(a):
    return escape(str(a))

In [76]:
@htmlize.register(Integral)
def html_integral_number(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

- Now, let's confirm that 10 is an integral number

In [77]:
isinstance(10, Integral)

True

In [78]:
htmlize(10)

'10'

- As we can see, our function didn't recognize 10 as an integral number
    - We need to fix this by using `isinstance` in our decorator